This configuration is used for core nodes similar to g4dn.12xlarge (48 vCPU and 192G RAM for each g4dn.12xlarge). You can modify the exectuor number, executor memory, cores per executor and memory overhead according to your cluster. 


In [1]:
%%configure -f
{
    "driverMemory": "10000M",
    "driverCores": 6,
    "executorMemory": "16000M",
    "conf" : {"spark.executor.instances":8, "spark.executor.cores": 6, "spark.task.cpus": 6, "spark.yarn.maxAppAttempts": 1, "spark.yarn.executor.memoryOverhead": "16G", "spark.sql.files.maxPartitionBytes": 4294967296, "spark.dynamicAllocation.enabled": false},
    "jars" : ["https://repo1.maven.org/maven2/ai/rapids/cudf/0.9.2/cudf-0.9.2.jar",
      "https://repo1.maven.org/maven2/ai/rapids/xgboost4j-spark_2.x/1.0.0-Beta5/xgboost4j-spark_2.x-1.0.0-Beta5.jar",
      "https://repo1.maven.org/maven2/ai/rapids/xgboost4j_2.x/1.0.0-Beta5/xgboost4j_2.x-1.0.0-Beta5.jar"]
}

In [2]:
sc.listJars.foreach(println)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1595883165900_0010,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

https://repo1.maven.org/maven2/ai/rapids/cudf/0.9.2/cudf-0.9.2.jar
spark://ip-172-31-2-164.ec2.internal:33595/jars/livy-core_2.11-0.7.0-incubating.jar
https://repo1.maven.org/maven2/ai/rapids/xgboost4j_2.x/1.0.0-Beta5/xgboost4j_2.x-1.0.0-Beta5.jar
spark://ip-172-31-2-164.ec2.internal:33595/jars/commons-codec-1.9.jar
spark://ip-172-31-2-164.ec2.internal:33595/jars/netty-all-4.1.17.Final.jar
https://repo1.maven.org/maven2/ai/rapids/xgboost4j-spark_2.x/1.0.0-Beta5/xgboost4j-spark_2.x-1.0.0-Beta5.jar
spark://ip-172-31-2-164.ec2.internal:33595/jars/livy-rsc-0.7.0-incubating.jar
spark://ip-172-31-2-164.ec2.internal:33595/jars/livy-repl_2.11-0.7.0-incubating.jar
spark://ip-172-31-2-164.ec2.internal:33595/jars/livy-api-0.7.0-incubating.jar


In [3]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1595883165900_0010,spark,idle,Link,Link,✔


In [4]:
// import notebook source
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.sql.types.{DoubleType, IntegerType, StructField, StructType}
import ml.dmlc.xgboost4j.scala.spark.{XGBoostClassifier, XGBoostClassificationModel}
//import ml.dmlc.xgboost4j.scala.spark.rapids.{GpuDataReader, GpuDataset}

import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.FloatType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.sql.types.{DoubleType, IntegerType, StructField, StructType}
import ml.dmlc.xgboost4j.scala.spark.{XGBoostClassifier, XGBoostClassificationModel}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.FloatType


In [5]:
//For AWS us-east-1 region, please use belowing S3 bucket
//For AWS us-west-2 region, please use DATA_PREFIX = "s3://spark-xgboost-mortgage-dataset-east1/csv/"

val DATA_PREFIX = "s3://spark-xgboost-mortgage-dataset-east1/csv/"
val trainPath = DATA_PREFIX + "train/2000*"
val evalPath  = DATA_PREFIX + "eval/2000*"


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DATA_PREFIX: String = s3://spark-xgboost-mortgage-dataset-east1/csv/
trainPath: String = s3://spark-xgboost-mortgage-dataset-east1/csv/train/2000*
evalPath: String = s3://spark-xgboost-mortgage-dataset-east1/csv/eval/2000*


In [6]:
sc.listJars.foreach(println)

val spark = SparkSession.builder.appName("mortgage-cpu").getOrCreate

//val dataReader = new GpuDataReader(spark)
val dataReader = spark.read

val labelColName = "delinquency_12"


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

https://repo1.maven.org/maven2/ai/rapids/cudf/0.9.2/cudf-0.9.2.jar
spark://ip-172-31-2-164.ec2.internal:33595/jars/livy-core_2.11-0.7.0-incubating.jar
https://repo1.maven.org/maven2/ai/rapids/xgboost4j_2.x/1.0.0-Beta5/xgboost4j_2.x-1.0.0-Beta5.jar
spark://ip-172-31-2-164.ec2.internal:33595/jars/commons-codec-1.9.jar
spark://ip-172-31-2-164.ec2.internal:33595/jars/netty-all-4.1.17.Final.jar
https://repo1.maven.org/maven2/ai/rapids/xgboost4j-spark_2.x/1.0.0-Beta5/xgboost4j-spark_2.x-1.0.0-Beta5.jar
spark://ip-172-31-2-164.ec2.internal:33595/jars/livy-rsc-0.7.0-incubating.jar
spark://ip-172-31-2-164.ec2.internal:33595/jars/livy-repl_2.11-0.7.0-incubating.jar
spark://ip-172-31-2-164.ec2.internal:33595/jars/livy-api-0.7.0-incubating.jar
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@11d17cf2
dataReader: org.apache.spark.sql.DataFrameReader = org.apache.spark.sql.DataFrameReader@5f21b85f
labelColName: String = delinquency_12


In [7]:
val schema = StructType(List(
    StructField("orig_channel", DoubleType),
    StructField("first_home_buyer", DoubleType),
    StructField("loan_purpose", DoubleType),
    StructField("property_type", DoubleType),
    StructField("occupancy_status", DoubleType),
    StructField("property_state", DoubleType),
    StructField("product_type", DoubleType),
    StructField("relocation_mortgage_indicator", DoubleType),
    StructField("seller_name", DoubleType),
    StructField("mod_flag", DoubleType),
    StructField("orig_interest_rate", DoubleType),
    StructField("orig_upb", IntegerType),
    StructField("orig_loan_term", IntegerType),
    StructField("orig_ltv", DoubleType),
    StructField("orig_cltv", DoubleType),
    StructField("num_borrowers", DoubleType),
    StructField("dti", DoubleType),
    StructField("borrower_credit_score", DoubleType),
    StructField("num_units", IntegerType),
    StructField("zip", IntegerType),
    StructField("mortgage_insurance_percent", DoubleType),
    StructField("current_loan_delinquency_status", IntegerType),
    StructField("current_actual_upb", DoubleType),
    StructField("interest_rate", DoubleType),
    StructField("loan_age", DoubleType),
    StructField("msa", DoubleType),
    StructField("non_interest_bearing_upb", DoubleType),
    StructField(labelColName, IntegerType)))



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

schema: org.apache.spark.sql.types.StructType = StructType(StructField(orig_channel,DoubleType,true), StructField(first_home_buyer,DoubleType,true), StructField(loan_purpose,DoubleType,true), StructField(property_type,DoubleType,true), StructField(occupancy_status,DoubleType,true), StructField(property_state,DoubleType,true), StructField(product_type,DoubleType,true), StructField(relocation_mortgage_indicator,DoubleType,true), StructField(seller_name,DoubleType,true), StructField(mod_flag,DoubleType,true), StructField(orig_interest_rate,DoubleType,true), StructField(orig_upb,IntegerType,true), StructField(orig_loan_term,IntegerType,true), StructField(orig_ltv,DoubleType,true), StructField(orig_cltv,DoubleType,true), StructField(num_borrowers,DoubleType,true), StructField(dti,DoubleType,...

In [8]:
// you can set max_depth to 8 and num_round to 100 to shorten the training time

val commParamMap = Map(
    "eta" -> 0.1,
    "gamma" -> 0.1,
    "missing" -> 0.0,
    "max_depth" -> 8,
    "max_leaves" -> 256,
    "grow_policy" -> "depthwise",
    "min_child_weight" -> 30,
    "lambda" -> 1,
    "scale_pos_weight" -> 2,
    "subsample" -> 1,
    "nthread" -> 6,
    "num_round" -> 100,
    "num_workers" -> 8,
    "tree_method" -> "hist")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

commParamMap: scala.collection.immutable.Map[String,Any] = Map(min_child_weight -> 30, grow_policy -> depthwise, scale_pos_weight -> 2, num_workers -> 8, subsample -> 1, lambda -> 1, max_depth -> 8, num_round -> 100, missing -> 0.0, tree_method -> hist, eta -> 0.1, max_leaves -> 256, gamma -> 0.1, nthread -> 6)


In [9]:

var (trainSet, evalSet) = {
  dataReader.option("header", true).schema(schema)
  (dataReader.csv(trainPath), dataReader.csv(evalPath))}

val featureNames = schema.filter(_.name != labelColName).map(_.name)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

trainSet: org.apache.spark.sql.DataFrame = [orig_channel: double, first_home_buyer: double ... 26 more fields]
evalSet: org.apache.spark.sql.DataFrame = [orig_channel: double, first_home_buyer: double ... 26 more fields]
featureNames: Seq[String] = List(orig_channel, first_home_buyer, loan_purpose, property_type, occupancy_status, property_state, product_type, relocation_mortgage_indicator, seller_name, mod_flag, orig_interest_rate, orig_upb, orig_loan_term, orig_ltv, orig_cltv, num_borrowers, dti, borrower_credit_score, num_units, zip, mortgage_insurance_percent, current_loan_delinquency_status, current_actual_upb, interest_rate, loan_age, msa, non_interest_bearing_upb)


In [10]:
object Vectorize {
  def apply(df: DataFrame, featureNames: Seq[String], labelName: String): DataFrame = {
    val toFloat = df.schema.map(f => col(f.name).cast(FloatType))
    new VectorAssembler()
      .setInputCols(featureNames.toArray)
      .setOutputCol("features")
      .transform(df.select(toFloat:_*))
      .select(col("features"), col(labelName))
  }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined object Vectorize


In [11]:
trainSet = Vectorize(trainSet, featureNames , labelColName)
evalSet = Vectorize(evalSet, featureNames , labelColName)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

trainSet: org.apache.spark.sql.DataFrame = [features: vector, delinquency_12: float]
evalSet: org.apache.spark.sql.DataFrame = [features: vector, delinquency_12: float]


In [12]:
object Benchmark {
  def time[R](phase: String)(block: => R): (R, Float) = {
    val t0 = System.currentTimeMillis
    val result = block // call-by-name
    val t1 = System.currentTimeMillis
    println("==> Benchmark: Elapsed time for [" + phase + "]: " + ((t1 - t0).toFloat / 1000) + "s")
    (result, (t1 - t0).toFloat / 1000)
  }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined object Benchmark


In [13]:
val modelPath = "/tmp/model"
val xgbClassifier = new XGBoostClassifier(commParamMap).setLabelCol(labelColName).setFeaturesCols(featureNames)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelPath: String = /tmp/model
xgbClassifier: ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier = xgbc_27c6d1357b6b


In [14]:
println("\n------ Training ------")
val (model, _) = Benchmark.time("train") {
        xgbClassifier.fit(trainSet)
}


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


------ Training ------
==> Benchmark: Elapsed time for [train]: 123.443s
model: ml.dmlc.xgboost4j.scala.spark.XGBoostClassificationModel = xgbc_27c6d1357b6b


In [15]:
// Save model if modelPath exists
model.write.overwrite().save(modelPath)
val xgbClassificationModel = model


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

xgbClassificationModel: ml.dmlc.xgboost4j.scala.spark.XGBoostClassificationModel = xgbc_27c6d1357b6b


In [16]:
println("\n------ Transforming ------")
val (results, _) = Benchmark.time("transform") {
  xgbClassificationModel.transform(evalSet)
}


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


------ Transforming ------
==> Benchmark: Elapsed time for [transform]: 0.381s
results: org.apache.spark.sql.DataFrame = [features: vector, delinquency_12: float ... 3 more fields]


In [17]:
println("\n------Accuracy of Evaluation------")
val evaluator = new MulticlassClassificationEvaluator().setLabelCol(labelColName)
val accuracy = evaluator.evaluate(results)
println(accuracy)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


------Accuracy of Evaluation------
evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_49c172a1ba9a
accuracy: Double = 0.985849170484196
0.985849170484196
